In [4]:
import pandas as pd, json, re

PATH = r"C:\Team2-final\JeonJungKyu\test2\perfume_final (1).csv"

try:
    df = pd.read_csv(PATH, encoding="utf-8-sig")
except UnicodeDecodeError:
    df = pd.read_csv(PATH, encoding="cp949")

unique = set()
for v in df["main_accords"].dropna().astype(str):
    v = v.strip()
    # JSON 배열로 파싱 시도
    try:
        arr = json.loads(v)
    except json.JSONDecodeError:
        # JSON이 아니면 구분자 정리 후 분리
        v = re.sub(r"[,\[\]\"]", " ", v)
        arr = [w for w in v.split() if w]
    unique.update(map(str.strip, arr))

print(sorted(unique))
print("총 개수:", len(unique))

# (선택) 파일로 저장
pd.Series(sorted(unique), name="unique_main_accord").to_csv(
    "unique_main_accords.csv", index=False, encoding="utf-8-sig"
)


['/', '그린', '너티', '라벤더', '락토닉', '럼', '레더', '로즈', '마린', '머스', '머스크', '머스키', '모스', '모시', '모씨', '바닐라', '바이올렛', '발사믹', '비즈왁스', '소트', '소프트', '솔티', '솝', '스모크', '스모키', '스위트', '스위티', '스트러스', '스파이', '스파이스', '스파이시', '시트러', '시트러스', '시패츌리', '시프러스', '아니스', '아로마', '아로마틱', '아몬드', '아이리스', '아쿠아틱', '알데하이드', '애니멀릭', '앰버', '얼', '얼디', '얼시', '얼씨', '엠버', '옐로우', '오드', '오우드', '오조닉', '우', '우드', '우디', '웜', '웜스파이스', '웜스파이시', '위스키', '윔', '체리', '카라멜', '카카오', '커피', '코코넛', '타바코', '토바코', '튜베로즈', '트로피칼', '파우더', '파우더리', '패출리', '패츌리', '페출리', '푸루티', '푸르티', '프레', '프레쉬', '프레시', '프루', '프루트', '프루티', '플라워', '플로', '플로랄', '플로럴', '허니', '허벌', '허브', '화이트', '화이트플로랄', '흘로랄']
총 개수: 93


In [5]:
import pandas as pd, json, re
from collections import Counter

# ★ CSV 경로 수정
PATH = r"C:\Team2-final\JeonJungKyu\test2\perfume_final (1).csv"

# 인코딩 자동 시도
for enc in ("utf-8-sig", "cp949"):
    try:
        df = pd.read_csv(PATH, encoding=enc)
        break
    except UnicodeDecodeError:
        continue

def parse_accords(cell):
    """JSON 배열 우선 파싱, 실패 시 구분자(콤마/공백/슬래시) 기준 분리"""
    if pd.isna(cell):
        return []
    text = str(cell).strip()
    # 1) JSON 배열 시도
    try:
        arr = json.loads(text)
        if isinstance(arr, list):
            return [str(x).strip() for x in arr if str(x).strip()]
    except Exception:
        pass
    # 2) 비JSON 문자열 fallback
    text = re.sub(r'[\[\]"]', '', text)
    parts = re.split(r'[,\s/]+', text)  # 콤마/공백/슬래시 분리
    return [p for p in parts if p]

# 전체 토큰 수집
all_acc = []
for v in df["main_accords"].fillna(""):
    all_acc.extend(parse_accords(v))

# 빈도 계산 → Top 10
cnt = Counter(all_acc)
top10 = cnt.most_common(10)

print("Top 10 main_accords:")
for name, c in top10:
    print(f"{name}: {c}")

# (선택) DataFrame으로 보고/저장
top10_df = pd.DataFrame(top10, columns=["main_accord", "count"])
top10_df["percent"] = (top10_df["count"] / len(all_acc) * 100).round(1)
print("\n----")
print(top10_df)

top10_df.to_csv("main_accords_top10.csv", index=False, encoding="utf-8-sig")
print("\nSaved -> main_accords_top10.csv")


Top 10 main_accords:
플로랄: 288
우디: 284
시트러스: 234
스파이시: 201
아로마틱: 177
화이트: 147
프레쉬: 142
파우더리: 110
로즈: 99
스위트: 97

----
  main_accord  count  percent
0         플로랄    288     10.7
1          우디    284     10.5
2        시트러스    234      8.7
3        스파이시    201      7.4
4        아로마틱    177      6.6
5         화이트    147      5.4
6         프레쉬    142      5.3
7        파우더리    110      4.1
8          로즈     99      3.7
9         스위트     97      3.6

Saved -> main_accords_top10.csv
